# Prediction

In [2]:
input_cardinality=20
output_cardinality=1
#change output_cardinality =[1,2,3,4,5,6,7,8,9,10,15,20]

In [3]:
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sklearn.model_selection as sk
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import joblib

In [4]:
#import pytorch_lightning as pl
import time
import copy
from tqdm import tqdm


In [5]:
torch.manual_seed(0)
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [6]:
#from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
#from pytorch_lightning.loggers import TensorBoardLogger

In [7]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [8]:
#pl.seed_everything(43)

In [9]:
class DATA:
    def __init__(self, data_path):
        self.data_path = data_path
        self.cols = 'ThetaXHG'
    def get_data(self):
        return self.read_data()
    
    def read_data(self): #polymorphism
        return pd.read_csv(self.data_path)

In [10]:
class THETADATASET(Dataset):
    #convert to pytorch dataset
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        sequence, output = self.X[idx], self.y[idx]
        return (torch.from_numpy(sequence.reshape(-1)), torch.from_numpy(output.reshape(-1)))
        #return dict(sequence=torch.tensor(sequence.reshape(-1),dtype=torch.float64), label=torch.tensor(output.reshape(-1),dtype=torch.float64))

In [11]:
DATA_PATH = "../DATASET/REALTIME_DATA/alldata.csv"
data = DATA(DATA_PATH)
samples = data.get_data()

In [14]:
samples[data.cols]

0      -0.001207
1       0.009104
2       0.005889
3       0.007015
4       0.011417
          ...   
1443    0.330298
1444    0.335613
1445    0.336424
1446    0.336424
1447    0.336741
Name: ThetaXHG, Length: 1448, dtype: float64

In [15]:
class LSTMNET(nn.Module):
    def __init__(self, batch_size, input_len, output_len, lstm_units = 4, num_layers=1):
        super(LSTMNET, self).__init__()
        self.input_len = input_len
        self.output_len = output_len
        self.num_layers = num_layers
        self.batch_size = batch_size
        #input_size = no of features = 1
        #hidden_size = no of lstm units in the layer
        #num_layers = no of lstm layers
        self.lstm_units = lstm_units
        self.lstm1 = nn.LSTM(input_size= 1, hidden_size= lstm_units, num_layers=num_layers,batch_first=True, dropout=0.6)
        
        self.linear0 = nn.Linear(in_features= 20, out_features=10)
        
        self.linear1 = nn.Linear(in_features= lstm_units, out_features=10)
        self.linear2 = nn.Linear(in_features= 10, out_features=10)
        self.ll = nn.Linear(in_features= 10, out_features=output_len)
        self.hidden = (torch.zeros(1*self.num_layers, self.batch_size, self.lstm_units).double(), torch.zeros(1*self.num_layers, self.batch_size, self.lstm_units).double())
        #print(self.hidden[0].device)
        #print(self.hidden.shape)
        #self.hidden[0]= self.hidden[0].to(device)
        #self.hidden[1] = self.hidden[1].to(device)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
    
    def reset_hidden_states(self,bs):
        self.hidden = (torch.zeros(1*self.num_layers, bs, self.lstm_units).double(), torch.zeros(1*self.num_layers, bs, self.lstm_units).double())
        
    def forward(self,x):
        #print(x.shape)
        #print(x)
        #print(x.unsqueeze(-1).shape)
        #print(self.hidden.shape)
        #print(self.hidden)
        lstm_out, (h,c) = self.lstm1(x.unsqueeze(-1), self.hidden)
        self.hidden= (h.detach(),c.detach())
        #c.detach_()
        #h.detach_()
        #self.hidden = (h.detach(), c.detach())
        #print(ht.shape)
        #ht=ht.to(device)
        #ct=ct.to(device)
        
        lstm_out = lstm_out[:,-1,:]
        #print(ht.shape)
        #either lstm_out goes to next or ht goes
        #lstm_out= h.detach()[-1]
        lin1_out = self.linear1(lstm_out)
        #Add RELU
        #lin0_out = F.relu(self.linear0(x))
        ll_out = self.ll(lin1_out)
        #x = self.linear0(x)
        #print(x.shape)
        
        #x = self.linear2(x)
        #Add RELU
        return ll_out

In [16]:
model_ckp_path = "../MODEL_CHECKPOINTS/model_lookahead"+str(output_cardinality)+".pth"
scaler_filename = "../SCALER_DUMPS/min_max_scaler_lookahead"+str(output_cardinality)+".save"
batch_size = 1
predictor = LSTMNET(batch_size=batch_size,input_len=input_cardinality,output_len=output_cardinality)
predictor.load_state_dict(torch.load(model_ckp_path))
#net2.eval()
predictor.reset_hidden_states(bs=1)
scaler = joblib.load(scaler_filename) 

predictor.eval() 
predictor = predictor.double()

/home/toor/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [17]:
#Generate predictions
j = input_cardinality
prediction_mismatch = list()
columns_= ['ThetaXHG_PREDICTION', 'ThetaXHG_GROUND']
for i in range(samples[data.cols].shape[0]-input_cardinality+1):
    X = samples[data.cols][i:i+input_cardinality]    
    if j <=samples[data.cols].shape[0]-1:
        y_ground = samples[data.cols][i+input_cardinality]
    else:
        y_ground = ''
    X_scaled = scaler.transform(X.to_numpy().reshape(X.shape[0],1))
    X_torch_scaled = torch.tensor(X_scaled.transpose(), dtype=torch.float64)
    y_pred_scaled = predictor(X_torch_scaled)
    y_pred = scaler.inverse_transform(y_pred_scaled.detach().numpy())
    prediction_mismatch.append([y_pred[0][output_cardinality-1],y_ground])
    j +=1

In [18]:
pd.DataFrame(prediction_mismatch, columns=columns_).to_csv('../RESULTS_PREDICTION/PREDICTION_lookahead'+str(output_cardinality)+".csv", index=False)

[0, 1, 2, 3, 4, 5, 6, 7, 8]